In [4]:
import os
import json
from dotenv import load_dotenv
from LLM_API import AuthenticatedRequestSender
from LLM_API import OpenAIService
from FileProcess import FileProcessor
from Memory import Embedding, LocalMemory
from Chat import JoiningChat
from WebExecuter import WebSearch
from FileProcess import CodeAnalyser
from Tools import FunctionManager
import re
import importlib
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import os
import whisper

analyser=CodeAnalyser()

dotenv_path = os.path.join(os.getcwd(), '.env')
root_path = dotenv_path.rstrip('.env')
load_dotenv(dotenv_path)

temp_sound_dir = os.path.join(root_path, "TempSound")
if not os.path.exists(temp_sound_dir):
    os.makedirs(temp_sound_dir)

# 录音文件路径
audio_file_path = os.path.join(temp_sound_dir, "temp_recording.wav")

# 预先加载Whisper模型
print("正在加载模型，请稍候...")
model = whisper.load_model("large")
print("模型加载完成。")

fileprocessor=FileProcessor(root_path)

project_root = root_path
function_manager = FunctionManager(project_root, service_type='sensetime')
service_type = 'sensetime'

100%|█████████████████████████████████████| 2.88G/2.88G [03:02<00:00, 16.9MiB/s]


模型加载完成。


In [9]:
def record_audio(file_path, fs=44100, channels=2):
    print("按回车开始录音，再次按回车结束录音。")
    input()  # 等待回车键开始
    
    # 准备录音
    recording = []
    def callback(indata, frames, time, status):
        # 将录音数据实时添加到列表中
        recording.append(indata.copy())

    # 开始录音
    print("正在录音...按回车键结束。")
    with sd.InputStream(callback=callback, samplerate=fs, channels=channels):
        input()  # 再次等待回车键结束录音
    
    print("录音结束。")

    # 将录音数据转换为NumPy数组，并保存为WAV文件
    recording_array = np.concatenate(recording, axis=0)
    wav.write(file_path, fs, np.int16(recording_array * 32767))
    
def transcribe_audio(model, file_path):
    # 使用预先加载的Whisper模型进行语音识别
    result = model.transcribe(file_path)
    return result["text"]
    
record_audio(audio_file_path)
question=transcribe_audio(model, audio_file_path)
print(question)

按回车开始录音，再次按回车结束录音。
正在录音...按回车键结束。
录音结束。
我想知道前200个斐波那奇数和素数的和


In [10]:
chat = JoiningChat(service_type=service_type, root_path=root_path)

ask = f"Based on the following function descriptions: '{function_manager.functions_info}', could you demonstrate how to address '{question}'? Please include a specific code snippet that performs this operation, assigns the result to a variable named 'result', and also assigns the name of the used function to a variable named 'used_function_name'. It is highly recommended to enclose your code within triple backticks and specify 'python' as the language, like so: ```python\n# Your code here\nresult = <operation>\nused_function_name = '<function_name>'\n```. This formatting helps ensure that your code can be correctly identified and processed."
answer=chat.process_input(ask)
print(answer)
# 使用正则表达式提取代码段
pattern = r"```python\n(.*?)\n```"
match = re.search(pattern, answer, re.DOTALL)

if match:
    original_code = match.group(1)
    code_string = re.sub(r"from [\w\.]+ import [\w\, ]+|import [\w\.]+", "", original_code, flags=re.MULTILINE)

    # 使用正则表达式提取需要导入的模块名称
    modules_to_import = ['Tools.tools']

    # 准备安全环境
    safe_globals = {
        "print": print,
        "__builtins__": {"print": print, "range": range},
    }
    
    def import_dependencies(dependencies, module):
        """
        导入函数的依赖。
        """
        for dep in dependencies:
            # 假设依赖的函数在同一个模块中
            if hasattr(module, dep):
                safe_globals[dep] = getattr(module, dep)
    # 动态导入模块并将函数添加到 safe_globals 中
    for function in function_manager.functions_info['functions']:
        file_path = function['file_path'].replace('\\', '.')  # 将文件路径转换为模块路径
        module_path = re.sub(r'\.py$', '', file_path)  # 移除.py扩展名
        module = importlib.import_module(module_path)  # 动态导入模块

        # 获取函数并添加到safe_globals
        func = getattr(module, function['name'])
        safe_globals[function['name']] = func
        
    # 导入函数的依赖
    if function['dependencies']:
        import_dependencies(function['dependencies'], module)
    # 执行代码
    try:
        exec(code_string, safe_globals)
        if 'result' in safe_globals and 'used_function_name' in safe_globals:
            execution_result = {"function_name": safe_globals['used_function_name'], "result": safe_globals['result']}
            print("Execution result:", execution_result)
        else:
            print("No result or used function name variable found in executed code.")
    except Exception as e:
        print("Error executing code:", e)

else:
    print("No code found.")

[系统]: 对话已命名为 '```python
from Tools.tools import prime_fib_sum

# 调用prime_fib_sum函数，计算前200个斐波那奇数和素数的和
result = prime_fib_sum(200)

# 将使用的函数名称赋值给used_function_name变量
used_function_name = 'prime_fib_sum'
```' 并保存。
```python
from Tools.tools import prime_fib_sum

# 调用prime_fib_sum函数，计算前200个斐波那奇数和素数的和
result = prime_fib_sum(200)

# 将使用的函数名称赋值给used_function_name变量
used_function_name = 'prime_fib_sum'
```
在这段代码中，我们首先导入了`Tools.tools`模块中的`prime_fib_sum`函数。然后，我们调用该函数，并将结果赋值给`result`变量。最后，我们将使用的函数名称赋值给`used_function_name`变量。
Execution result: {'function_name': 'prime_fib_sum', 'result': 453973694165307953197296969697410619345412}


In [11]:
def prime_fib_sum(n):
    def is_prime(num):
        if num <= 1:
            return False
        for i in range(2, int(num**0.5) + 1):
            if num % i == 0:
                return False
        return True

    count, num, sum_primes = 0, 2, 0
    while count < n:
        if is_prime(num):
            sum_primes += num
            count += 1
        num += 1

    if n <= 0:
        sum_fibs = 0
    else:
        fib1, fib2 = 0, 1
        sum_fibs = fib1 + fib2
        for _ in range(2, n):
            fib_next = fib1 + fib2
            sum_fibs += fib_next
            fib1, fib2 = fib2, fib_next

    return sum_primes + sum_fibs
prime_fib_sum(200)

453973694165307953197296969697410619345412